In [48]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime, timezone, timedelta
from kafka import KafkaConsumer
from minio import Minio
from minio.error import S3Error
from io import BytesIO

In [40]:
# Carrega as variáveis de ambiente do arquivo .env localizado no diretório atual
env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path=env_path)

# Recupera as credenciais e informações necessárias do ambiente
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
tenant_id = os.getenv('TENANT_ID')
subscription_id = os.getenv('SUBSCRIPTION_ID')

In [41]:
# Obtém o horário atual em UTC
agora_utc = datetime.now(timezone.utc)

# Formata a data/hora atual para o padrão ISO sem os segundos (ex: '2025-07-11T18:17')
data_atual = agora_utc.strftime('%Y-%m-%dT%H:%M')

# Calcula o horário de dois minutos atrás
dois_minutos_atras = agora_utc - timedelta(minutes=2)

# Formata a data/hora de dois minutos atrás
data_dois_minutos_atras = dois_minutos_atras.strftime('%Y-%m-%dT%H:%M')

In [42]:
# Monta a URL para obter o token de autenticação do Azure AD
auth_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
auth_data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "resource": "https://management.azure.com/"
}

# Monta a URL para consultar os custos na API do Azure Cost Management
costs_url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.CostManagement/query?api-version=2021-10-01"
headers = {"Content-Type": "application/json"}

In [43]:
# Define o corpo da consulta para buscar os custos agrupados por Resource Group e ResourceId, no intervalo dos últimos 2 minutos
query_body = {
    "type": "Usage",
    "timeframe": "Custom",
    "timePeriod": {
        "from": f"{data_dois_minutos_atras}:00Z",  # início do intervalo
        "to": f"{data_atual}:59Z"                   # fim do intervalo
    },
    "dataset": {
        "granularity": "Daily",
        "grouping": [
            {"type": "Dimension", "name": "ResourceGroup"},
            {"type": "Dimension", "name": "ResourceId"}
        ],
        "aggregation": {
            "totalCost": {
                "name": "PreTaxCost",
                "function": "Sum"
            }
        }
    }
}

In [44]:
response = requests.post(auth_url, data=auth_data)
response.raise_for_status()  # Lança exceção se houver erro HTTP
token = response.json()["access_token"]
headers["Authorization"] = f"Bearer {token}"  # Adiciona o token ao header

# Faz a requisição para obter os dados de custo do Azure
response = requests.post(costs_url, headers=headers, json=query_body)
response.raise_for_status()
cost_data = response.json()

print("Dados de custo extraídos com sucesso.")

HTTPError: 429 Client Error: Too Many Requests for url: https://management.azure.com/subscriptions/da483b95-1caf-404c-bfe4-36abef87f6e6/providers/Microsoft.CostManagement/query?api-version=2021-10-01

In [45]:
# Verifica se a resposta possui os dados esperados
if cost_data and "properties" in cost_data and "rows" in cost_data["properties"] and "columns" in cost_data["properties"]:
    columns = [col["name"] for col in cost_data["properties"]["columns"]]
    print(columns)
    idx_data = columns.index('UsageDate')
    documents_to_send = []
    # Para cada linha de dados, cria um dicionário associando coluna e valor
    for row in cost_data["properties"]["rows"]:
        row[idx_data] = data_atual
        print(row)
        document = dict(zip(columns, row))
        documents_to_send.append(document)

['PreTaxCost', 'UsageDate', 'ResourceGroup', 'ResourceId', 'Currency']
[0.00756989439480001, '2025-07-15T00:08', 'nintendodatabrickswi86no-workspace-rg', '/subscriptions/da483b95-1caf-404c-bfe4-36abef87f6e6/resourcegroups/nintendodatabrickswi86no-workspace-rg/providers/microsoft.storage/storageaccounts/dbstorage7ifgyhjijpdgi', 'BRL']
[0.0590212689300001, '2025-07-15T00:08', 'nintendoproject', '/subscriptions/da483b95-1caf-404c-bfe4-36abef87f6e6/resourcegroups/nintendoproject/providers/microsoft.storage/storageaccounts/nintendostorageaccount', 'BRL']
[0.0, '2025-07-15T00:08', 'nintendoproject', '/subscriptions/da483b95-1caf-404c-bfe4-36abef87f6e6/resourcegroups/nintendoproject/providers/microsoft.web/sites/appfunckabum', 'BRL']
[0.0, '2025-07-15T00:08', 'nintendoproject', '/subscriptions/da483b95-1caf-404c-bfe4-36abef87f6e6/resourcegroups/nintendoproject/providers/microsoft.web/sites/appfuncmagalu', 'BRL']


In [ ]:
# Configuração do consumidor Kafka para ouvir o tópico 'api-topic'
consumer = KafkaConsumer(
    'api-topic',  # Nome do tópico Kafka
    bootstrap_servers=['kafka1:19091'],  # Endereço do servidor Kafka
    value_deserializer=lambda v: json.loads(v.decode('utf-8')),  # Deserializa as mensagens recebidas de JSON
    auto_offset_reset='latest',  # Começa a consumir a partir da última mensagem disponível
    enable_auto_commit=True  # Confirma automaticamente o offset das mensagens lidas
)

In [ ]:
for message in consumer:
    data = message.value
    print("Mensagem recebida:", data)
    usage_date = data['UsageDate']
    usage_date = usage_date.replace("-", "_").replace("T", "_").replace(":", "_")
    resource = data['ResourceId']
    resource = resource.rsplit('/', 1)[-1]

    break

KeyboardInterrupt: 

In [49]:
# Converter JSON para bytes
json_bytes = json.dumps(data).encode('utf-8')
json_stream = BytesIO(json_bytes)

In [50]:
client = Minio(
    "minio:9000",
    access_key=os.getenv('MINIO_ROOT_USER'),
    secret_key=os.getenv('MINIO_ROOT_PASSWORD'),
    secure=False
)


bucket_name = "azurecost"
object_name = f"inbound/{resource}_{usage_date}.json"

# Crie o bucket se não existir
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)

# Enviar diretamente da memória
client.put_object(
    bucket_name=bucket_name,
    object_name=object_name,
    data=json_stream,
    length=len(json_bytes),
    content_type="application/json"
)

print(f"{object_name} enviado com sucesso para o bucket {bucket_name}")

inbound/nintendostorageaccount_2025_07_14_23_48.json enviado com sucesso para o bucket azurecost
